In [3]:
import pandas as pd
import requests
from datetime import datetime, date, timedelta
from pytz import timezone

In [5]:
host = '127.0.0.1'
schema = 'gans_local'
user = 'root'
password = 'Mysql_key'
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [24]:
airports_df = pd.read_csv("airport_list.csv")
airports_df

,icao,iata,name,shortName,municipalityName,countryCode
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE
1,EDDH,HAM,Hamburg,Hamburg,Hamburg,DE
2,EDDW,BRE,Bremen,Bremen,Bremen,DE
3,EDDM,MUC,Munich,Munich,Munich,DE
4,EDDK,CGN,Cologne Bonn,Bonn,Cologne,DE
5,EDDL,DUS,Duesseldorf Düsseldorf,Düsseldorf,Duesseldorf,DE
6,EDLW,DTM,Dortmund,Dortmund,Dortmund,DE
7,EDLV,NRN,Weeze Niederrhein,Niederrhein,Weeze,DE
8,EDDF,FRA,Frankfurt-am-Main,Frankfurt-am-Main,Frankfurt-am-Main,DE


In [12]:
querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"true"}
headers = {
	"X-RapidAPI-Key": "flights_key",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

In [18]:
today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
tomorrow = (today + timedelta(days=1))

In [19]:
times = [["00:00","11:59"],["12:00","23:59"]]

In [25]:
icao_list = airports_df['icao'].to_list()

In [26]:
icao_list

['EDDB', 'EDDH', 'EDDW', 'EDDM', 'EDDK', 'EDDL', 'EDLW', 'EDLV', 'EDDF']

In [6]:
# Flight Funtion

In [ ]:
def flight_function(times, icao_list, tomorrow, headers, querystring, flights_df):
    # First loop
    for time in times:
        for icao in icao_list[1:2]:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
            response = requests.request("GET", url, headers=headers, params=querystring)
            print(response)
            flight_json = response.json()
    
    # Second loop
    for time in times:
        for icao in icao_list:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
            response = requests.request("GET", url, headers=headers, params=querystring)
            print(response)
            flight_json = response.json()

            for flight in flight_json['flights']:  # Assuming the key 'flights' holds the list of flights in the json response
                flights_df['icao_id'].append(icao)
                flights_df['arrival_terminal'].append(flight['arrival'].get('terminal'))
                scheduled_time = flight['arrival']['scheduledTime'].get('local', None)
                revised_time = flight['arrival'].get('revisedTime', {}).get('local', scheduled_time)
                flights_df['arrival_local_time'].append(revised_time)
                flights_df['arrival_from_where'].append(flight['departure']['airport'].get('name', None))
                flights_df["flight_number"].append(flight.get("number", None))
                flights_df["flight_status"].append(flight.get("status", None))
                aircraft_model = flight.get('aircraft', {}).get('model', None)
                flights_df["airplane_model"].append(aircraft_model)
                flights_df["airline_name"].append(flight['airline'].get("name", None))
                depart_icao = flight['departure']['airport'].get('icao', None)
                flights_df["depart_icao"].append(depart_icao)

    return flights_df

# Usage example:
# Assuming all required variables like 'times', 'icao_list', etc. are defined
# flights_df = flight_function(times, icao_list, tomorrow, headers, querystring, flights_df)


In [14]:
flights_df

{'icao_id': [],
 'arrival_terminal': [],
 'arrival_local_time': [],
 'arrival_from_where': [],
 'flight_number': [],
 'flight_status': [],
 'airplane_model': [],
 'airline_name': [],
 'depart_icao': []}

In [15]:
flights_df = pd.DataFrame(flights_dict)

NameError: name 'flights_dict' is not defined

In [23]:
flights_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ICAO               371 non-null    object
 1   departure_airport  371 non-null    object
 2   local_time         371 non-null    object
dtypes: object(3)
memory usage: 8.8+ KB


In [24]:
flights_df['local_time'] = pd.to_datetime(flights_df['local_time'])

In [27]:
flights_check = pd.read_sql('flights',con=con)
flights_check

,flight_id,ICAO,departure_airport,local_time
0,1,EGKK,LTAI,2023-10-06 00:55:00
1,2,EGKK,GCRR,2023-10-06 00:45:00
2,3,EGKK,LEAL,2023-10-06 00:10:00
3,4,EGKK,LGKO,2023-10-06 00:05:00
4,5,EGKK,LTBS,2023-10-06 00:40:00
...,...,...,...,...
366,367,EGKK,LEPA,2023-10-06 21:20:00
367,368,EGKK,LEBL,2023-10-06 21:05:00
368,369,EGKK,LCLK,2023-10-06 21:15:00
369,370,EGKK,LEMG,2023-10-06 21:55:00


In [25]:
flights_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype                               
---  ------             --------------  -----                               
 0   ICAO               371 non-null    object                              
 1   departure_airport  371 non-null    object                              
 2   local_time         371 non-null    datetime64[ns, pytz.FixedOffset(60)]
dtypes: datetime64[ns, pytz.FixedOffset(60)](1), object(2)
memory usage: 8.8+ KB


In [26]:
flights_df.to_sql('flights',con=con,if_exists='append',index=False)

371

In [35]:
import requests
from datetime import datetime, timedelta
from pytz import timezone

# settings
today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
tomorrow = (today + timedelta(days=1))
times = [["00:00","11:59"],["12:00","23:59"]]

# flights_df initialization
flights_df = {
    'icao_id': [],
    'arrival_terminal': [],
    'arrival_local_time': [],
    'arrival_from_where': [],
    'flight_number': [],
    'flight_status': [],
    'airplane_model': [],
    'airline_name': [],
    'depart_icao': []
}

# Looping through times and icao_list
for time in times:
    for icao in icao_list:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
        try:
            response = requests.request("GET", url, headers=headers, params=querystring)
            response.raise_for_status()  # will raise an HTTPError if the HTTP request returned an unsuccessful status code
            flight_json = response.json()
            
            # Extract flights from flight_json
            flights = flight_json.get('data', [])
            
            # Looping through flights to populate flights_df
            for flight in flights:
                flights_df['icao_id'].append(icao)
                flights_df['arrival_terminal'].append(flight['arrival'].get('terminal'))
                scheduled_time = flight['arrival']['scheduledTime'].get('local', None)
                revised_time = flight['arrival'].get('revisedTime', {}).get('local', scheduled_time)
                flights_df['arrival_local_time'].append(revised_time)
                flights_df['arrival_from_where'].append(flight['departure']['airport'].get('name', None))
                flights_df["flight_number"].append(flight.get("number", None))
                flights_df["flight_status"].append(flight.get("status", None))
                aircraft_model = flight.get('aircraft', {}).get('model', None)
                flights_df["airplane_model"].append(aircraft_model)
                flights_df["airline_name"].append(flight['airline'].get("name", None))
                depart_icao = flight['departure']['airport'].get('icao', None)
                flights_df["depart_icao"].append(depart_icao)

SyntaxError: incomplete input (2630712801.py, line 49)